<a href="https://colab.research.google.com/github/JieShenAI/torch/blob/main/huggingface/example/bbc_news_classification/bbc_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "drive/MyDrive/github/torch/example/data/"
model_path = "drive/MyDrive/model/"

## 导包

In [ ]:
# BBC 文章分类
# 数据集来源: https://www.kaggle.com/competitions/learn-ai-bbc/
# 代码参考: https://mp.weixin.qq.com/s/00on_zUFjAmnoSb_8j0QMw

import torch
from torch import nn
from transformers import BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import pandas as pd
from torch.optim import Adam
from tqdm import tqdm
import time

In [ ]:
labels = {
    'business': 0,
    'entertainment': 1,
    'sport': 2,
    'tech': 3,
    'politics': 4
}

label_name = {v: k for k, v in labels.items()}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = {
    'batch': 2,
    'epochs': 4,
    'device': device,
    'bert_path': 'bert-base-cased',
    'train_path': data_path + 'BBC News Train.csv',
    'test_path': data_path + 'BBC News Test.csv',
    'model_path': model_path + 'bert_bbc_class.pth'
}
device = config['device']

## 数据集

In [ ]:
class SampleDataset(Dataset):
    def __init__(self, df,
                 is_train=True,
                 tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')):
        self.is_train = is_train
        if is_train:
            self.labels = [labels[label] for label in df['Category']]
        self.texts = [tokenizer(text,
                      padding='max_length',
                      max_length=512,
                      truncation=True,
                      return_tensors="pt")
                      for text in df['Text']]

    def classes(self):
        assert self.is_train
        return self.labels

    def __len__(self):
        return len(self.texts)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        assert self.is_train
        # return torch.tensor([self.labels[idx]])
        return self.labels[idx]
        # TODO: 没有转成tensor, 但dataloder中的label是tensor

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        if not self.is_train:
            return self.get_batch_texts(idx)
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y



def get_dataset():
  df = pd.read_csv(config['train_path'])
  dataset = SampleDataset(df)
  # torch切分数据集
  trainset, validset, testset = random_split(dataset, [0.8, 0.1, 0.1])
  return trainset, validset, testset

## 模型

In [ ]:
# class BertClassifier(nn.Module):
#     def __init__(self, dropout=0.5):
#         super().__init__()
#         self.bert = BertModel.from_pretrained('bert-base-cased')
#         self.dropout = nn.Dropout(dropout)
#         self.linear1 = nn.Linear(768, 256)
#         self.linear2 = nn.Linear(256, 5)
#         self.relu = nn.ReLU()

#     # def forward(self, input_id, mask):
#     def forward(self, input):
#         output = self.bert(**input)
#         pooler_output = output.pooler_output
#         dropout_output = self.dropout(pooler_output)
#         linear_output = self.linear1(dropout_output)
#         layer = self.relu(linear_output)
#         return self.linear2(layer)


## 训练

In [ ]:
trainset, validset, testset = get_dataset()

In [ ]:
train_dataloader = DataLoader(trainset, batch_size=config['batch'], shuffle=True, pin_memory=True)
val_dataloader = DataLoader(validset, batch_size=config['batch'], shuffle=False, pin_memory=True)

In [ ]:
# for feature, label in train_dataloader:
#   label = label.to(device)
#   # for k,v in feature.items():
#   #   print(k,v.shape)
#   input = {k: v.squeeze(1).to(device) for k, v in feature.items()}
#   v = model(**input,labels=label)
#   loss = v['loss']
#   logits = v['logits']
#   print(loss)
#   print(logits.shape)
#   break

In [ ]:
len(train_dataloader), len(val_dataloader)

(596, 75)

In [ ]:
def run_one_epoch(model, dataloader, is_train, optimizer=None, scheduler=None, batch=config['batch']):
    def run():
      start = time.time()
      total_loss = 0
      total_acc = 0
      nums = len(dataloader) * batch
      for feature, label in tqdm(dataloader):
        label = label.to(device)
        if is_train:
          optimizer.zero_grad()
        input = {k: v.squeeze(1).to(device) for k, v in feature.items()}
        output = model(**input,labels=label)
        loss = output['loss']
        logits = output['logits']
        if is_train:
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()
        total_loss += loss.item()
        acc = (logits.argmax(dim=-1) == label.view(-1)).sum().item()
        total_acc += acc
      end = time.time()
      print(end - start)
      return total_loss / nums, total_acc / nums

    if is_train:
        model.train()
        return run()
    if not is_train:
        model.eval()
        with torch.no_grad():
            return run()


def train(model, train_dataset,
          val_dataset,
          optimizer=None,
          scheduler=None,
          batch=config['batch'],
          epochs=config['epochs'],
          device=device):
    # DataLoader根据batch_size获取数据，训练时选择打乱样本


    min_loss = float('inf')
    for epoch_num in range(epochs):
        train_loss, train_acc = run_one_epoch(model, train_dataloader, True, optimizer, scheduler)
        val_loss, val_acc = run_one_epoch(model, val_dataloader, False)
        print(f'Epoch: {epoch_num + 1}')
        print(f'Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}')
        print(f'Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}')

        new_loss = 0.8*train_loss + 0.2*val_loss
        if new_loss < min_loss:
          min_loss = new_loss
          print(epoch_num, "save model")
          torch.save(model.state_dict(), config['model_path'])


def predict(model, device=device, batch=config['batch']):
    # 加载模型
    model.load_state_dict(torch.load(config['model_path']))
    model.eval()
    test_dataset = SampleDataset(pd.read_csv(config['test_path']), is_train=False)
    # 加载测试集
    test_dataloader = DataLoader(test_dataset, batch_size=batch, shuffle=False)
    tensors = []
    model.eval()
    with torch.no_grad():
        for features in tqdm(test_dataloader):
            mask = features['attention_mask'].to(device)
            input_id = features['input_ids'].squeeze(1).to(device)
            output = model(input_id, mask)
            y_pred = output.argmax(dim=1)
            tensors.append(y_pred.view(-1))
    t = torch.concat(tensors, dim=-1).tolist()
    ans = [label_name[key] for key in t]
    return ans

In [ ]:
# model = BertClassifier().to(device)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
          lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
          eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
          )

total_steps = 4 * len(train_dataloader)

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps = 0, # Default value in run_glue.py
                        num_training_steps = total_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train(model, trainset, validset, optimizer, scheduler)

100%|██████████| 596/596 [02:15<00:00,  4.39it/s]


135.64366722106934


100%|██████████| 75/75 [00:05<00:00, 14.61it/s]


5.142120122909546
Epoch: 1
Train loss: 0.1932, Train acc: 0.8826
Val loss: 0.0747, Val acc: 0.9600
0 save model


100%|██████████| 596/596 [02:15<00:00,  4.40it/s]


...

137.87937903404236


100%|██████████| 75/75 [00:05<00:00, 14.62it/s]


5.136518239974976
Epoch: 5
Train loss: 0.0033, Train acc: 0.9983
Val loss: 0.0066, Val acc: 0.9867
4 save model


  9%|▉         | 56/596 [00:12<02:03,  4.36it/s]


KeyboardInterrupt: ignored

In [ ]:
device

device(type='cuda')

## test

In [ ]:
# 在测试集上看看训练的结果如何
def predict_demo(model, dataset):
  model.eval()
  test_dataloader = DataLoader(dataset, batch_size=2, shuffle=False, pin_memory=True)
  total_acc = 0
  total_num = 0
  with torch.no_grad():
    for feature, label in test_dataloader:
      label = label.to(device)
      input = {k: v.squeeze(1).to(device) for k, v in feature.items()}
      output = model(**input)
      logits = output['logits']
      total_num += label.size(0)
      y_hat = logits.argmax(dim=-1).to(device)
      total_acc += torch.sum(y_hat == label).item()
  return total_acc / total_num
      
predict_demo(model,testset)

0.9865771812080537

发现当epoch=3时，效果最好，故最后在整个训练集上再train一次

注意，在最后一次训练时，必须重新对 model, optim重新初始化

In [ ]:
best_epochs = 3

In [ ]:
df = pd.read_csv(config['train_path'])
all_dataset = SampleDataset(df)
finnal_dataloader = DataLoader(all_dataset, batch_size=config['batch'], shuffle=True, pin_memory=True)

In [ ]:
def finnal_train(model, best_epochs, dataloader, optimizer, scheduler, batch=config['batch']):
  model.train()
  for epoch_num in range(best_epochs):
    start = time.time()
    total_loss = 0
    total_acc = 0
    nums = len(dataloader) * batch
    for feature, label in tqdm(dataloader):
      label = label.to(device)
      optimizer.zero_grad()
      input = {k: v.squeeze(1).to(device) for k, v in feature.items()}
      output = model(**input,labels=label)
      loss = output['loss']
      logits = output['logits']
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
      total_loss += loss.item()
      acc = (logits.argmax(dim=-1) == label.view(-1)).sum().item()
      total_acc += acc
    end = time.time()
    print(end - start)
    train_loss = total_loss / nums
    train_acc = total_acc / nums

    print(f'Epoch: {epoch_num + 1}')
    print(f'Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}')
  
  torch.save(model.state_dict(), config['model_path'])

In [ ]:
finnal_train(model, best_epochs, finnal_dataloader, optimizer, scheduler)

100%|██████████| 745/745 [02:45<00:00,  4.49it/s]


165.90052604675293
Epoch: 1
Train loss: 0.1392, Train acc: 0.9262


100%|██████████| 745/745 [02:44<00:00,  4.52it/s]


164.73130750656128
Epoch: 2
Train loss: 0.0400, Train acc: 0.9866


100%|██████████| 745/745 [02:46<00:00,  4.48it/s]


166.40321612358093
Epoch: 3
Train loss: 0.0142, Train acc: 0.9953


In [ ]:
def predict(model, device=device, batch=config['batch']):
  # 加载模型
  model.load_state_dict(torch.load(config['model_path']))
  model.eval()
  test_dataset = SampleDataset(pd.read_csv(config['test_path']), is_train=False)
  # 加载测试集
  test_dataloader = DataLoader(test_dataset, batch_size=batch, shuffle=False)
  model.eval()
  ans = []
  with torch.no_grad():
    for feature in test_dataloader:
      input = {k: v.squeeze(1).to(device) for k, v in feature.items()}
      output = model(**input)
      logits = output['logits']
      ans += logits.argmax(dim=-1).tolist()
  ans = [label_name[key] for key in ans]
  return ans

In [ ]:
ans = predict(model)

In [ ]:
ans

In [ ]:
sample_csv = data_path + 'BBC News Sample Solution.csv'
sample_df = pd.read_csv(sample_csv)
sample_df.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,business
3,459,entertainment
4,1020,politics


In [ ]:
len(ans)

735

In [ ]:
sample_df.shape

(735, 4)

In [ ]:
sample_df.iloc[:,1] = ans
sample_df.head()

,ArticleId,Category,-1,1
0,1018,sport,sport,sport
1,1319,tech,tech,tech
2,1138,sport,sport,sport
3,459,business,business,business
4,1020,sport,sport,sport


In [ ]:
all(sample_df['Category'] == ans)

True

In [ ]:
sample_df = sample_df.iloc[:,:2]
sample_df.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,sport
3,459,business
4,1020,sport


In [ ]:
all(sample_df['Category'] == ans)

True

In [ ]:
sample_df.to_csv("submission.csv",index=False)